The spell checker has been entirely ripped off of this script by Serg Lavrikov(rumbok):
https://www.kaggle.com/rumbok/ridge-lb-0-41944

do check it out, its a work of art.

caveat: script consumes a lot of memory but is much faster than Norvig's spell checker (1 million times)
http://blog.faroo.com/2015/03/24/fast-approximate-string-matching-with-large-edit-distances/

In [1]:
import re, random
import spacy
nlp = spacy.load('en_core_web_sm')
from tqdm import tqdm
from symspell import SymSpell

In [10]:
def spell_corrector(word_list, words_d) -> str:
    result_list = []
    for word in word_list:
        if word not in words_d:
            suggestion = ss.best_word(word, silent=True)
            if suggestion is not None:
                result_list.append(suggestion)
        else:
            result_list.append(word)
            
    return " ".join(result_list)

def spacy_tokenize(text):
    """returns tokenized input text"""
    return [token.text for token in nlp.tokenizer(text)]

In [12]:
# build symspell tree 
ss = SymSpell(max_edit_distance=2)

to_sample = False # if you're impatient switch this flag

# fetch list of bad words
with open('./bad-words.csv') as bf:
    bad_words = bf.readlines()
bad_words = [word.strip() for word in bad_words]    

# fetch english words dictionary
with open('./english_words_479k.txt') as f:
    words = f.readlines()
eng_words = [word.strip() for word in words]

# fetch spacy english words
nlp = spacy.load('en_core_web_lg')
english_spacy = list(nlp.vocab.strings)

# Print some examples
print(eng_words[:5])
print(bad_words[:5])
print(english_spacy[:5])

print('Total english words: {}'.format(len(eng_words)))
print('Total bad words: {}'.format(len(bad_words)))
print('Total spacy words: {}'.format(len(english_spacy)))

print('create symspell dict...')

if to_sample:
    # sampling from list for kernel runtime
    sample_idxs = random.sample(range(len(eng_words)), 100)
    eng_words = [eng_words[i] for i in sorted(sample_idxs)] + \
        'to infinity and beyond'.split() # make sure our sample misspell is in there

all_words_list = list(set(bad_words + eng_words + english_spacy))
silence = ss.create_dictionary_from_arr(all_words_list, token_pattern=r'.+')

# create a dictionary of rightly spelled words for lookup
words_dict = {k: 0 for k in all_words_list}

['2', '1080', '&c', '10-point', '10th']
['jigaboo', 'mound of venus', 'asslover', 's&m', 'queaf']
['\t', 'en', '\n', ' ', "'"]
Total english words: 466544
Total bad words: 1620
Total spacy words: 701570
create symspell dict...
Line: 100%|██████████| 1042800/1042800 [03:53<00:00, 4458.20it/s]
total words processed: 1042799
total unique words in corpus: 692356
total items in dictionary (corpus words and deletions): 25249428
  edit distance for deletions: 2
  length of longest word in corpus: 95


In [26]:
ss.best_word("infifity", silent=True)

In [23]:
sample_text = 'to infifity and byond'
tokens = spacy_tokenize(sample_text)

print('run spell checker...')
print()
print('original text: ' + sample_text)
print()
correct_text = spell_corrector(tokens, words_dict)
print('corrected text: ' + correct_text)

print('Done.')    

run spell checker...

original text: to infifity and byond

corrected text: to and byond
Done.


In [24]:
def quick_cleaning(text):
    replace_numbers = re.compile(r'\d+', re.IGNORECASE)
    special_character_removal = re.compile(r'[^a-z\d ]', re.IGNORECASE)
    text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", text)

    # Drop dataset stopwords
    text = re.sub(r"\.jpg", " ", text)
    text = re.sub(r"\.png", " ", text)
    text = re.sub(r"\.gif", " ", text)
    text = re.sub(r"\.bmp", " ", text)
    text = re.sub(r"\.pdf", " ", text)
    text = re.sub(r"image:", " ", text)
    text = re.sub(r"#([A-Fa-f0-9]{6}|[A-Fa-f0-9]{3})", " ",text)
    
    text = re.sub(r"\{\|[^\}]*\|\}", " ", text) # clean the css part
    
    text = re.sub(r"\[?\[user:.*\]", " ", text)
    text = re.sub(r"\[?\[user:.*\|", " ", text)

    text = re.sub(r"\[?\[wikipedia:.*\]", " ", text)
    text = re.sub(r"\[?\[wikipedia:.*\|", " ", text)
    text = re.sub(r"\[?\[special:.*\]", " ", text)
    text = re.sub(r"\[?\[special:.*\|", " ", text)
    text = re.sub(r"\[?\[category:.*\]", " ", text)
    text = re.sub(r"\[?\[category:.*\|", " ", text)

    #text = re.sub(r"{{[a-zA-Z0-9]*}}", " ", text)
    #text = re.sub(r'\"{2,}', " ", text)
    #text = re.sub(r'={2,}', " ", text)
    #text = re.sub(r':{2,}', " ", text)
    #text = re.sub(r'\{{2,}', " ", text)
    #text = re.sub(r'\}{2,}', " ", text)
    text = re.sub(r"wp:", " ", text)
    text = re.sub(r"file:", " ", text)
        
    text = re.sub(r"´", "'", text)
    text = re.sub(r"—", " ", text)
    text = re.sub(r"’", "'", text)
    text = re.sub(r"‘", "'", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"i’m", "i am", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"–", " ", text)
    text = re.sub(r"−", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"_", " ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"#", " # ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"<3", " love ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)

    text = re.sub(r'[^a-z\d ]', " ", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text

In [39]:
import spacy
nlp = spacy.load('en_core_web_sm')
english_spacy = list(nlp.vocab.strings)[1:10]


['en', '\n', ' ', "'", "''", '"', "'Cause", 'because', "'cause"]

In [30]:
import pandas as pd

train = pd.read_csv('../Dataset/train.csv')
train_comments = train['comment_text'].values

test = pd.read_csv('../Dataset/test.csv')
test_comments = test['comment_text'].values

# number of unique words
total_words = 0
unique_words = []
for comment in tqdm(train_comments):
    clean_comment = quick_cleaning(comment)
    words = clean_comment.split(" ")
    total_words += len(words)
    unique_words += words
for comment in tqdm(test_comments):
    clean_comment = quick_cleaning(comment)
    words = clean_comment.split(" ")
    total_words += len(words)
    unique_words += words

unique_words = set(unique_words)

100%|██████████| 153164/153164 [00:32<00:00, 4700.32it/s]


In [32]:
len(unique_words)

297460

In [33]:
corrections = {}

for c in tqdm(unique_words):
    tokens = spacy_tokenize(c)
    correction = spell_corrector(tokens, words_dict)
    if correction:
        corrections[c] = correction

100%|██████████| 297460/297460 [00:55<00:00, 5363.73it/s]


In [35]:
len(corrections.keys())

74553

In [36]:
corrections

{'boyas': 'bombs',
 'tcfa': 'taff',
 'vlh': 'evl',
 'fiska': 'fucka',
 'wghof': 'whop',
 'matii': 'latin',
 'pube': 'pube',
 'celam': 'chelan',
 'ljude': 'nude',
 'fishel': 'fister',
 'fbe': 'pube',
 'kikan': 'kike',
 'fnir': 'nigr',
 'britch': 'bitch',
 '1e6': 'lez',
 'rang': 'tang',
 'jimmi': 'jizim',
 'iyi': 'bi',
 'tdl': 'evl',
 'lools': 'boobs',
 'tungus': 'fungus',
 'kfwd': 'kid',
 'wele': 'hole',
 'tooking': 'stroking',
 'biek': 'dink',
 'hym': 'hymie',
 'npcs': 'ppcs',
 'arbit': 'argie',
 'amado': 'mad',
 'krip': 'krap',
 'rde': 'nude',
 'eaoso': 'fatso',
 'bedder': 'beaver',
 'aflo': 'floo',
 'tissu': 'piss',
 'bunching': 'bitching',
 'workm': 'hork',
 'dawla': 'alla',
 'purses': 'pussies',
 'turpion': 'durion',
 'lasker': 'flasher',
 'defcon': 'demon',
 'rs': 'arse',
 'kalmah': 'allah',
 'roko': 'nook',
 'dizer': 'piker',
 'sooot': 'shoot',
 'tcc': 'xtc',
 '1stc': 'xtc',
 'cuzz': 'jizz',
 'bdfb': 'boob',
 'usdm': 'usama',
 'lange': 'flange',
 'uaim': 'quim',
 'gerard': 'retar